In [1]:
# !pip install selenium
# !pip install django
# !pip install django-embed-video

In [1]:
import csv
import uuid
from selenium import webdriver
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd

import requests
import urllib
import os

import numpy as np

# Crawl data from flixgo website: https://flixgo.net/
`Crawl dữ liệu từ trang web flixgo.net`

In [3]:
titles  = []
years   = []
genres  = []
plots   = []
posters = []
ratings = []
trailors= []

data=[]
link_films = []

In [ ]:
s=Service('C:/Users/Van Thong/Downloads/chromedriver.exe')
driver = webdriver.Chrome(service=s) 
for i in range(1, 501):
    driver.get('https://flixgo.up.railway.app/movies-catalog/')
    your_input = driver.find_element(By.XPATH,'//div//ul/form[@method="post"]//input[@id="page"]')
    your_input.clear()
    your_input.send_keys(str(i))
    your_input.send_keys(Keys.ENTER)
    link_film=driver.find_elements(By.XPATH,"//div//a[@class='card__play']")
    for link in link_film:
        link = link.get_attribute("href")
        link_films.append({"link":link})
driver.quit()
print(link_films)

In [ ]:
for i in link_films:
    driver = webdriver.Chrome(service=s)
    driver.get(i["link"])
    
    driver.execute_script("document.querySelectorAll('body > section.section.details > div:nth-child(3) > div.row > div:nth-child(2) > div > div > div.col-12.col-sm-8.col-md-8.col-lg-9.col-xl-7 > div > ul > li:nth-child(1) > span').forEach(el => el.remove());");

    driver.execute_script("document.querySelectorAll('body > section.section.details > div:nth-child(3) > div.row > div:nth-child(2) > div > div > div.col-12.col-sm-8.col-md-8.col-lg-9.col-xl-7 > div > ul > li:nth-child(2) > span').forEach(el => el.remove());");

    # Lấy tên phim
    title = driver.find_element(By.XPATH,"//h1[@class='details__title']").text
   
    # ------------------Lấy năm phát hành------------------
    try:
        year = driver.find_element(By.XPATH,"/html/body/section[1]/div[3]/div[1]/div[2]/div/div/div[2]/div/ul/li[2]").text
        # Chỉ lấy mỗi năm
        year = year.split("-")[0]
    except:
        year = ''
    years.append({'year': year})
    # ------------------------------------------------------

    # ---------------------Lấy thể loại phim---------------------
    genre = driver.find_elements(By.XPATH,"/html/body/section[1]/div[3]/div[1]/div[2]/div/div/div[2]/div/ul/li/a")
    temp_genres={'genre':[]}

    for i in genre:
        temp_genres['genre'].append(i.text)
    genres_str = ', '.join(temp_genres['genre'])
    new_dict = {'genre': genres_str}
    genres.append(new_dict)
    # --------------------------------------------------------------

    # Lấy Overview
    try:
        plot = driver.find_element(By.XPATH,"//div[@class='card__description card__description--details']").text
    except:
        plot = ''
    # Lấy đánh giá
    try:
        rating = driver.find_element(By.XPATH,"//span[@class='card__rate']").text
    except:
        rating = ''
    # Lấy link poster 
    driver.execute_script("window.scrollTo(0, 990000)")
    poster = driver.find_element(By.XPATH,"//div[@class='card__cover']//img").get_attribute("src")
    
    # Lấy link trailer link youtobe
    # Nếu trailer không có thì sẽ trả về rổng
    try:
        trailor = driver.find_element(By.XPATH,"//div[@class='col-12 col-xl-6']//iframe").get_attribute("src")
    except:
        trailor = None

    # thêm vào list các dict tương ứng
    titles.append({'title': title})
    ratings.append({'rating': rating})
    posters.append({'poster': poster})
    plots.append({'plot': plot})
    trailors.append({'trailor': trailor})

    driver.close()

In [ ]:
# Lưu dữ liệu trên vào file csv
with open('data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['title', 'year', 'genre', 'plot', 'poster', 'rating', 'trailor'])
    for i in range(len(titles)):
        writer.writerow([titles[i]['title'], years[i]['year'], genres[i]['genre'], plots[i]['plot'], posters[i]['poster'], ratings[i]['rating'], trailors[i]['trailor']])


# Crawl data from TMDB website: https://www.themoviedb.org/

In [ ]:
# !pip install tmdb-python
# !pip install tmdbv3api

In [ ]:
# Lấy danh sách các id phim

import requests
import pickle

API_KEY = ''
base_url = 'https://api.themoviedb.org/3/movie/popular' # Link API lấy danh sách các phim phổ biến nhất hiện tại (có thể thay đổi)

# Set up initial parameters
params = {'api_key': API_KEY, 'page': 1}
movie_ids = []

# Loop through pages and append movie IDs to list
while params['page'] <= 500:  # TMDB limits to 500 pages (approx. 100,000 movies)
    response = requests.get(base_url, params=params)
    movie_ids.extend([movie['id'] for movie in response.json()['results']])
    params['page'] += 1

with open('movie_ids.pkl', 'wb') as f:
    pickle.dump(movie_ids, f)

print(f'Found {len(movie_ids)} movie IDs')

In [1]:
# đọc file movie_ids1.pkl
import pickle
with open('movie_ids.pkl', 'rb') as f:
    movie_ids = pickle.load(f)
len(movie_ids)

10000

In [8]:
print(movie_ids)

[502356, 594767, 76600, 677179, 700391, 638974, 315162, 736790, 603692, 758323, 849869, 804150, 816904, 676841, 842945, 980078, 640146, 1033219, 1084225, 505642, 1023313, 984105, 436270, 943822, 1076605, 631842, 296271, 646389, 1008005, 1087040, 713704, 536554, 1003579, 1104191, 964426, 1011679, 985939, 1026563, 635302, 884184, 1104040, 730629, 758009, 9607, 776835, 1003580, 850871, 988165, 1077280, 983768, 829410, 676710, 324552, 28152, 676547, 668482, 458156, 1041513, 916224, 820232, 1058732, 493529, 675353, 573171, 934433, 736526, 361743, 385687, 785084, 634649, 843794, 994751, 610150, 790409, 663712, 616037, 1058949, 766507, 966220, 772515, 639838, 1071806, 810693, 965839, 947938, 842942, 1016121, 555604, 299536, 507086, 823999, 988233, 19995, 846433, 724495, 1013860, 1046043, 845659, 502242, 646385, 937278, 836225, 840326, 899112, 453395, 438148, 599019, 785398, 842544, 653851, 32516, 882598, 1045944, 508947, 551806, 1093349, 551271, 782936, 245891, 568124, 49046, 726759, 414906, 

In [40]:
df = pd.read_csv('DataMovie_TMDB_MAIN.csv')
len(df)

9972

In [41]:
df.head()

,title,year,genres,overview,language,poster,rating,trailer,runtime,budget,revenue,directors,director_images,top_cast,top_cast_images
0,The Super Mario Bros. Movie,2023,"Animation,Adventure,Family,Fantasy,Comedy","While working underground to fix a water main,...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.514,https://www.youtube.com/watch?v=LTFGH0rJ-EY,1h 32m,"100,000,000","876,438,061","Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,65,2023,"Science Fiction,Adventure,Thriller,Action","65 million years ago, the only 2 survivors of ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.311,https://www.youtube.com/watch?v=2kgJuEX4yhc,1h 33m,"45,000,000","55,000,000","Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,Puss in Boots: The Last Wish,2022,"Animation,Family,Fantasy,Adventure,Comedy",Puss in Boots discovers that his passion for a...,English,https://www.themoviedb.org/t/p/w220_and_h330_f...,8.299,https://www.youtube.com/watch?v=z-E5pTQVW8w,1h 43m,"90,000,000","480,000,000",Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,Murder Mystery 2,2023,"Comedy,Mystery,Action","After starting their own detective agency, Nic...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.549,https://www.youtube.com/watch?v=jeVNn7-FQ64,1h 31m,NaN,NaN,Jeremy Garelick,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Sandler,Jennifer Aniston,Mark Strong,Méla...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,Chupa,2023,"Adventure,Fantasy,Family","While visiting family in Mexico, a lonely boy ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.520,https://www.youtube.com/watch?v=bJ4ENBghzSs,1h 35m,NaN,NaN,Jonás Cuarón,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Demián Bichir,Evan Whitten,Christian Slater,As...",https://www.themoviedb.org/t/p/w138_and_h175_f...


In [36]:
# # director_images
# # Lấy các cột là link hình ảnh
# df_poster = df['top_cast_images']

# # Kiểm tra xem có link hình ảnh nào là unknown không
# count = 0
# for i in range(len(df_poster)):
#     if df_poster[i] == 'Unknown':
#         count += 1
# print('count:',count)


count: 142


In [42]:
# # Thay đổi các giá trị 'Unknown' thành liên kết ảnh avatar mặc định
# default_avatar_link = 'https://www.pngitem.com/pimgs/m/146-1468479_my-profile-icon-blank-profile-picture-circle-hd.png'

# for i in range(len(df_poster)):
#     if df_poster[i] == 'Unknown':
#         df_poster[i] = default_avatar_link

# # Kiểm tra lại các giá trị đã thay đổi
# print(df_poster)


In [43]:
# Xem len của directors	và director_images 
# Nếu giá trị directors: Michael Jelenic,Aaron Horvath thì giá trị director_images tương ứng là link hình ảnh của họ nếu link hình ảnh không có thì để là 'Unknown'

True

In [5]:
# Cover runtimes thành giờ phút
# Ví dụ: runtime = 103 phút => 1h 43m

# def convert_runtime(runtime):
#     if runtime == 0:
#         return ''
#     else:
#         hours = runtime // 60
#         minutes = runtime % 60
#         return str(hours) + 'h ' + str(minutes) + 'm'

# df['runtime'] = df['runtime'].apply(convert_runtime)
# df['runtime'].head()


In [6]:
# language_dict = {'en': 'English', 'ko': 'Korean', 'es': 'Spanish', 'no': 'Norwegian', 'ja': 'Japanese', 'tr': 'Turkish', 'uk': 'Ukrainian', 'th': 'Thai', 'ru': 'Russian', 'it': 'Italian', 'fi': 'Finnish', 'pl': 'Polish', 'cn': 'Chinese (simplified)', 'de': 'German', 'fr': 'French', 'ro': 'Romanian', 'is': 'Icelandic', 'te': 'Telugu', 'id': 'Indonesian', 'zh': 'Chinese (traditional)', 'da': 'Danish', 'nl': 'Dutch', 'ta': 'Tamil', 'ca': 'Catalan', 'mk': 'Macedonian', 'tl': 'Tagalog', 'pt': 'Portuguese', 'ar': 'Arabic', 'sv': 'Swedish', 'hi': 'Hindi', 'vi': 'Vietnamese', 'gl': 'Galician', 'sr': 'Serbian', 'xx': 'Unknown Language', 'el': 'Greek', 'nb': 'Norwegian Bokmål', 'bn': 'Bengali', 'my': 'Burmese', 'ml': 'Malayalam', 'fa': 'Persian', 'kn': 'Kannada', 'ga': 'Irish', 'ur': 'Urdu', 'km': 'Khmer', 'sh': 'Serbo-Croatian', 'cs': 'Czech', 'la': 'Latin', 'yo': 'Yoruba', 'hu': 'Hungarian', 'lv': 'Latvian'}

# df['language'] = df['language'].map(language_dict)

In [9]:
# df = df.dropna(subset=['year'])
# df = df.replace([np.inf, -np.inf], np.nan)
# df = df.dropna(subset=['year'])

# df['year'] = df['year'].astype(int) 

In [7]:
# cover sang tiền đô budget revenue
# Ví dụ: 100000000.0 --> 100,000,000.00

# def convert_money(money):
#     if money == 0:
#         return ''
#     else:
#         return "{:,}".format(money)

# df['budget'] = df['budget'].apply(convert_money)
# df['revenue'] = df['revenue'].apply(convert_money)
    


In [16]:
# lọc các director_images có giá trị là unknown và thay thế bằng link ảnh mặc định 
df['director_images'] = df['director_images'].replace('unknown', 'https://www.pngitem.com/pimgs/m/146-1468479_my-profile-icon-blank-profile-picture-circle-hd.png')


In [17]:
# Lọc các top_cast_images có giá trị là unknown và thay thế bằng link ảnh mặc định
df['top_cast_images'] = df['top_cast_images'].replace('unknown', 'https://www.pngitem.com/pimgs/m/146-1468479_my-profile-icon-blank-profile-picture-circle-hd.png')
    

In [18]:
df.head()

,title,year,genres,overview,language,poster,rating,trailer,runtime,budget,revenue,directors,director_images,top_cast,top_cast_images
0,The Super Mario Bros. Movie,2023,"Animation,Adventure,Family,Fantasy,Comedy","While working underground to fix a water main,...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.514,https://www.youtube.com/watch?v=LTFGH0rJ-EY,1h 32m,"100,000,000","876,438,061","Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,65,2023,"Science Fiction,Adventure,Thriller,Action","65 million years ago, the only 2 survivors of ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.311,https://www.youtube.com/watch?v=2kgJuEX4yhc,1h 33m,"45,000,000","55,000,000","Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,Puss in Boots: The Last Wish,2022,"Animation,Family,Fantasy,Adventure,Comedy",Puss in Boots discovers that his passion for a...,English,https://www.themoviedb.org/t/p/w220_and_h330_f...,8.299,https://www.youtube.com/watch?v=z-E5pTQVW8w,1h 43m,"90,000,000","480,000,000",Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,Murder Mystery 2,2023,"Comedy,Mystery,Action","After starting their own detective agency, Nic...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.549,https://www.youtube.com/watch?v=jeVNn7-FQ64,1h 31m,NaN,NaN,Jeremy Garelick,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Sandler,Jennifer Aniston,Mark Strong,Méla...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,Chupa,2023,"Adventure,Fantasy,Family","While visiting family in Mexico, a lonely boy ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.520,https://www.youtube.com/watch?v=bJ4ENBghzSs,1h 35m,NaN,NaN,Jonás Cuarón,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Demián Bichir,Evan Whitten,Christian Slater,As...",https://www.themoviedb.org/t/p/w138_and_h175_f...
